In [ ]:
!pip install librosa
!pip install --upgrade sklearn

In [6]:
#The output of this section is the CSV files with the data to be handle by the model
CREATE_CSV_FILES = True

In [7]:
# Defines the names of the CSV files
TRAIN_CSV_FILE = "Speaker_Train_File.csv"
TEST_CSV_FILE = "Speaker_Test_File.csv"

In [3]:
import matplotlib.pyplot as plt
import scipy.io.wavfile
import numpy as np
from matplotlib import cm
import pandas as pd
import librosa
import zipfile as zf
import csv
import os
from sklearn import metrics
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
from sklearn import tree
# import zipfile as zf
from sklearn import svm
from sklearn.mixture import GaussianMixture

In [ ]:
def extractWavFeatures(soundFilesFolder, csvFileName):
    print("The features of the files in the folder "+soundFilesFolder+" will be saved to "+csvFileName)
    header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
    for i in range(1, 41):
        header += f' mfcc{i}'
    header += ' label'
    header = header.split()
#     print('CSV Header: ', header)
    file = open(csvFileName, 'w', newline='')
    #with file:
    writer = csv.writer(file)
    writer.writerow(header)
    for filename in os.listdir(soundFilesFolder):
        if filename.endswith('.wav'):
            number = f'{soundFilesFolder}/{filename}'
            y, sr = librosa.load(number, mono=True)
            # remove leading and trailing silence
            y, index = librosa.effects.trim(y)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 40)
            to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
            for e in mfcc:
                to_append += f' {np.mean(e)}'
            writer.writerow(to_append.split())
    file.close()
    print("End of extractWavFeatures")

In [ ]:
if (CREATE_CSV_FILES == True):
    extractWavFeatures("Audio_Data/Speaker_recognition_train_data", TRAIN_CSV_FILE)
    extractWavFeatures("Audio_Data/Speaker_recognition_test_data", TEST_CSV_FILE)
    print("CSV files are created")
else:
    print("CSV files creation is skipped")

In [4]:
#Reading a dataset and convert file name to corresbonding number
def preProcessData(csvFileName):
    header_name_list = ['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'mfcc21', 'mfcc22', 'mfcc23', 'mfcc24', 'mfcc25', 'mfcc26', 'mfcc27', 'mfcc28', 'mfcc29', 'mfcc30', 'mfcc31', 'mfcc32', 'mfcc33', 'mfcc34', 'mfcc35', 'mfcc36', 'mfcc37', 'mfcc38', 'mfcc39', 'mfcc40', 'label']
    print(csvFileName+ " will be preprocessed")
    data =  pd.read_csv(csvFileName)
#     data = pd.read_csv(csvFileName, skiprows=[1, 50]
    # we have 4 speakers: 
    # 0: Marina
    # 1: Mohab
    # 2: Omnia
    # 3: Youssef
    filenameArray = data['filename'] 
    speakerArray = []
#     print(filenameArray)
    for filename in filenameArray:
        #print(speaker)
        if "marina" in filename:
            speaker = 0
        elif "mohab" in filename:
            speaker = 1
        elif "yousef" in filename:
            speaker = 2
        elif "omnia" in filename:
            speaker = 3
        else: 
            speaker = 4
#         print(speaker)
        speakerArray.append(speaker)
    data['number'] = speakerArray
    #Dropping unnecessary columns
    data = data.drop(['filename'],axis=1)
    data = data.drop(['label'],axis=1)
    data = data.drop(['chroma_stft'],axis=1)
#     data.shape
#     print("Preprocessing is finished")
#     print(data[['filename', 'number']])
#     print(data)
    return data

In [8]:
trainData = preProcessData(TRAIN_CSV_FILE)
testData = preProcessData(TEST_CSV_FILE)

Speaker_Train_File.csv will be preprocessed
Speaker_Test_File.csv will be preprocessed


In [12]:
X_train = np.array(trainData.iloc[:, :-1], dtype = float)
y_train = trainData.iloc[:, -1]
X_test = np.array(testData.iloc[:, :-1], dtype = float)
y_test = testData.iloc[:, -1]

print("Y from training data:", y_train.shape)
print("Y from test data:", y_test.shape)

Y from training data: (238,)
Y from test data: (55,)


In [13]:
DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train)

DecisionTreeClassifier()

In [14]:
test_pred_decision_tree = DTC.predict(X_test)
test_pred_decision_tree

array([0, 1, 1, 4, 4, 0, 1, 3, 2, 2, 1, 3, 4, 3, 2, 2, 1, 4, 2, 0, 2, 0,
       2, 3, 1, 0, 3, 0, 4, 4, 1, 2, 2, 2, 0, 1, 1, 4, 2, 1, 2, 3, 3, 1,
       3, 4, 3, 0, 0, 4, 0, 1, 1, 1, 2])

In [25]:
input_audio = 'Audio_Data/open_test2.wav'
audio_features = extract_features_from_input(input_audio)
audio_features = audio_features.reshape(1,45)

In [29]:
print("Accuracy_dtr1:",metrics.accuracy_score(y_test, test_pred_decision_tree))
# print(test_pred_decision_tree)
# print( y_test)

Accuracy_dtr1: 0.8545454545454545


In [23]:
def extract_features_from_input(input_audio):
    extracted_features = []
    y, sr = librosa.load(input_audio, mono=True, duration=30)
    y, index = librosa.effects.trim(y)
#     chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 40)
    extracted_features.extend([np.mean(rmse), np.mean(spec_cent),np.mean(spec_bw), np.mean(rolloff), np.mean(zcr)])
    for e in mfcc:
        extracted_features.append(np.mean(e))
    return np.array(extracted_features)

In [27]:
prediction = DTC.predict(np.array(audio_features))
speakers_list = [(0, 'Marina'), (1, 'Mohab'), (2, 'Yousef'), (3, 'Omnia'), (4, 'Others')]
for iterator, speaker in enumerate(speakers_list):
    if prediction[0] == speaker[0]:
        print("voice of " + speaker[1])


voice of Omnia


In [18]:
# save the model to disk
import pickle
filename = 'speech_classifier_final.sav'
pickle.dump(DTC, open(filename, 'wb'))
 

In [36]:
gmm = GaussianMixture(n_components=4, covariance_type='full').fit(X_train)
prediction_gmm = gmm.predict(X_test)
print("Accuracy_GMM:",metrics.accuracy_score(y_test, prediction_gmm))

Accuracy_GMM: 0.38181818181818183


## 